# Traffic Sign Identification using Deep Neural Networks

## Description
Lorem Ipsum

## Setup and Image Preprocessing

### Imports

In [1]:
import cv2
import pickle
import csv

#A tua mina, tem ganda vagina
#Mas não foste tu quem escavou bro, foi o Chico da Tina

### Preprocessing Function
Resizes and Normalizes all pictures in all of our datasets and then stores them in a dictionary.

In [2]:
def preprocess_images(file_name):
    path = './Dataset/'   # Change this for the path corresponding to your base Dataset directory

    img_infos = {'labels':[],'images':[]}

    with open(path+file_name,"r") as csv_file:
        reader = csv.reader(csv_file, delimiter=',', quotechar='|')
        next(reader)
        
        for row in reader:

            img = cv2.imread(path+row[-1], cv2.IMREAD_UNCHANGED)    
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Convert image to Grayscale
            
            img = cv2.resize(img, (32,32),interpolation = cv2.INTER_AREA) #Resize the image to 32x32

            #img = cv2.addWeighted( img, 1.2, img, 0, 0) #Increase contrast
            
            cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX) #Normalize
            
            img_infos['images'].append(img) #Save Image pixels
            img_infos['labels'].append(row[-2]) #Save image Label

    return img_infos

### Save treated data as Pickle
Srsly, funniest thing I've ever seen

In [3]:
img_infos = preprocess_images("Test.csv")
with open("./Dataset/PickledData/Test.p", 'wb') as pickle_rick:
    pickle.dump(img_infos,pickle_rick)
    
img_infos = preprocess_images("Train.csv")
with open("./Dataset/PickledData/Train.p", 'wb') as pickle_rick:
    pickle.dump(img_infos,pickle_rick)

## Neural Network

### Step 0 - Imports

In [25]:
import random
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras

from sklearn.utils import shuffle

### Step 1 - Get the data into the sets

In [26]:
with open("./Dataset/PickledData/Test.p", "rb") as f:
    test = pickle.load(f)
with open("./Dataset/PickledData/Train.p", "rb") as f:
    train = pickle.load(f)

# Get the training set
x_train, y_train = train['images'], train['labels']

# Get the cross validation set
x_cv = []
y_cv = []
for i in range(int(0.2*len(x_train))): # Randomly move 20% of the training examples into the cross-validation set
    index = random.randint(0, len(x_train)-1)
    x_cv.append(x_train.pop(index))
    y_cv.append(y_train.pop(index))
    
# Get the test set
x_test, y_test = test['images'], test['labels']

# Convert all our sets into numPy Arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

x_cv = np.array(x_cv)
y_cv = np.array(y_cv)

x_test = np.array(x_test)
y_test = np.array(y_test)

### Step 2 - Treat the data (Normalize and Shuffle)

In [27]:
# Normalize the data sets (Subtract and divide by half of the range of values. Gets the values between -1 and 1)
x_train = ((x_train - 127.5) / 127.5)
x_cv = ((x_cv - 127.5) / 127.5)
x_test = ((x_test - 127.5) / 127.5)

# Shuffle the training set
x_train, y_train = shuffle(x_train, y_train)

### Step 3 - Define the number of Epochs and each Batch's size

In [28]:
EPOCHS = 50
BATCH_SIZE = 200

### Step 4 - Define the Neural Network

In [33]:
def LeNetModel(no_output=6, rgb=True):
    input_shape = (32,32,3) if rgb else (32,32,1) # Define the image's size
    
    model = keras.Sequential() # Define the Keras Model
    
    # First Layer - Convolutional ; Input = 32x32x3 for RGB , 32x32x1 for GrayScale ; Output = 28x28x6
    model.add(keras.layers.Conv2D(
                filters=no_output, kernel_size=(3, 3),
                activation='relu', input_shape=input_shape
                )
             )
    
    # Second Layer - Subsampling (Average Pooling) ; Input = 28x28x6 ; Output = 14x14x6
    model.add(keras.layers.AveragePooling2D())
    
    # Third Layer - Convolutional ; Input = 14x14x6 ; Output = 10x10x16
    model.add(keras.layers.Conv2D(
                    filters=16, kernel_size=(3, 3),
                    activation='relu'
                    )
                 )
    
    # Fourth Layer - Subsampling (Average Pooling) ; Input = 10x10x16 ; Output = 5x5x16
    model.add(keras.layers.AveragePooling2D())
    
    model.add(keras.layers.Flatten()) # Flatten the last layer's output to pass it to the Fully Connected layers
    
    # Fifth Layer - Fully Connected ; Input = 5x5x16 ; Output = 120x1
    model.add(keras.layers.Dense(units=120, activation='relu'))

    # Sixth Layer - Fully Connected ; Input = 5x5x16 ; Output = 120x1
    model.add(keras.layers.Dense(units=84, activation='relu'))
    
    # Output Layer - Output = 43x1
    model.add(keras.layers.Dense(units=43, activation = 'softmax'))
    
    return model

### Step 5 - Create the model and define evaluation

In [35]:
model = LeNetModel()

lr = 0.002 # Define the learning rate to be used by our optimizer
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=lr),
              metrics=['accuracy'])

In [7]:
rate = 0.002

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

ValueError: Shape must be rank 4 but is rank 3 for 'Conv2D' (op: 'Conv2D') with input shapes: [?,32,32], [5,5,1,6].

In [10]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
prediction = tf.argmax(logits, 1)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(x_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = x_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(x_cv, y_cv)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...



ValueError: Cannot feed value of shape (200, 32, 32) for Tensor 'Placeholder:0', which has shape '(?, 32, 32, 3)'

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(x_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))